In [2]:
import torch
from torch import nn

import numpy as np
import pandas as pd
from scipy.stats import norm
from urllib.request import urlopen

from torch.distributions.constraints import positive

import pyro
import pyro.distributions as dist
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam
from pyro.contrib.oed.eig import marginal_eig
from pyro.nn.module import to_pyro_module_
import pyro.poutine as poutine
from pyro.contrib.util import lexpand
from pyro.nn import PyroModule, PyroSample, PyroParam

import os
import sys
parent_dir_abs = os.path.abspath(os.pardir)
sys.path.insert(0, parent_dir_abs) 
from pyro_oed_src import posterior_eig

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx
import math

In [3]:
device = torch.device("cuda") if torch.cuda.is_available() else "cpu"
#device = "cpu"
print(f'Using device: {device}.')

Using device: cuda.


In [4]:
# thresholds for measuring data
designs = [0,2,4,6,8,10]

In [12]:
torch.poisson(torch.tensor(5.0, device=device))

tensor(2., device='cuda:0')

In [51]:
lam_0 = torch.tensor(0.0, device=device)
lam_1 = torch.tensor(10.0, device=device)
n = torch.tensor(1000, device=device)

def model(d):
    samples = []
    with pyro.plate("plate", n): 
        theta = pyro.sample("theta", dist.Uniform(lam_0,lam_1)).to(device)
        m = pyro.sample("m", dist.Poisson(theta)).to(device)
        samples.append(m)
    return torch.sum(torch.stack(samples) >= d)

In [35]:
pyro.get_param_store()['y']

tensor(5061., device='cuda:0', requires_grad=True)